# Loading a graph in Python

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from time import time
from itertools import islice


np.set_printoptions(legacy='1.25')

## Load the input file

We consider a citation graph in which nodes are researchers and edges indicate how many times one researcher cited another, so this is a weighted, directed graph. First, we upload the file with `Pandas` and convert it into a dataframe.

We name the four columns `i`, `j`, `w`. The first two, `i` and `j`, are the indeces associated with nodes. The third, `w` is the weight of each edge.

In [2]:
df = pd.read_csv('../Data/citation.edges')
df.head()

,i,j,w
0,0x1,0x1007,1
1,0x1,0x100b,1
2,0x1,0x100d,1
3,0x1,0x1066,1
4,0x1,0x1084,1


## Preprocess the graph format

### Nodes' id

Now we preprocess the nodes' identities. This operation is not always necessary but it is a good custom. It can happen that the nodes' identities inside a file are aribtrary numbers, codes or even names. The most convienent thing to do is to work with node identities that are integers between 0 and n-1 (we are using Python!), where $n$ is the number of nodes. 

In [3]:
# exctract all nodes that appear in the network
all_nodes = np.unique(df[['i', 'j']])
print(f'all_nodes: {all_nodes}')

all_nodes: ['0x1' '0x10' '0x100' ... '0xffd' '0xffe' '0xfff']


In [4]:
# now we extract the number of nodes
n = len(all_nodes)
print(f'The graph has {n} nodes')

# we then create a mapping between the nodes' identities and integers
NodeMapper = dict(zip(all_nodes, np.arange(n)))

# print the first 15 elements
dict(islice(NodeMapper.items(), 15))

The graph has 28093 nodes


{'0x1': 0,
 '0x10': 1,
 '0x100': 2,
 '0x1000': 3,
 '0x1001': 4,
 '0x1002': 5,
 '0x1003': 6,
 '0x1004': 7,
 '0x1005': 8,
 '0x1006': 9,
 '0x1007': 10,
 '0x1008': 11,
 '0x1009': 12,
 '0x100a': 13,
 '0x100b': 14}

In [5]:
# Now we perform the mapping
df.i = df.i.map(lambda x: NodeMapper[x])
df.j = df.j.map(lambda x: NodeMapper[x])
df.head()

,i,j,w
0,0,10,1
1,0,14,1
2,0,16,1
3,0,111,1
4,0,143,1


## Managing network files

Now that we have the file in the proper format, we will create graphs in Python in two ways: using the `NetworkX` library meant for graph analysis and using only sparse adjacency matrices for our analysis.

### NetworkX

> **Note**: when we get the adjacency matrix, we should also use the sparse format. This allows us to effectively operate with a matrix, but actually only store the edge list, without the need of occpuying memory space with the (many) zero elements of the matrix.

In [7]:
# this creates a weighted and directed graph using NetworkX
t0 = time()

g_wd = nx.DiGraph()
g_wd.add_weighted_edges_from(df.values)
A_wd = nx.adjacency_matrix(g_wd)

# this creates a directed graph using igraph
g_d = nx.DiGraph()
g_d.add_edges_from(df[['i', 'j']].values)
A_d = nx.adjacency_matrix(g_d)

# this creates an undirected graph using igraph
g = nx.Graph()
g.add_edges_from(df[['i', 'j']].values)
A = nx.adjacency_matrix(g)

print(f'Execution time: {time() - t0}')

Execution time: 26.987175703048706


In [8]:
# check 1: the matrix A_wd actually has weights in it
print(f'The weights in the matrix A_wd are: {np.unique(A_wd[A_wd.nonzero()])}')

The weights in the matrix A_wd are: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  79  80  81  82  83  84  85  86  88  89  90  91  93
  94  96  97  98 101 102 103 105 110 112 114 116 121 127 129 136 145 151
 157 163 195 197 202 262]


In [9]:
# check 2: the matrix A_d is not symmetric
print(f'The matrix A_d - A_d.T has {(A_d - A_d.T).nnz} non-zero elements')

The matrix A_d - A_d.T has 6296894 non-zero elements


In [10]:
# Check 3: the matrix A is symmetric
print(f'The matrix A - A.T has {(A - A.T).nnz} non-zero elements')

The matrix A - A.T has 0 non-zero elements


### Build the sparse matrix directly from the edge list

Passing through a Python package to create the adjacency matrix might not be the most convenient thing to do in terms of computational time. Let us see how one can build the sparse adjacency matrix directly from the dataframe `df`.

In [11]:
t0 = time()

# weighted and directed
A_wd_ = csr_matrix((df.w, (df.i, df.j)), shape = (n,n))

# unweighted and directed
A_d_ = A_wd_.sign()

# unweighted and undirected
A_ = (A_d_ + A_d_.T).sign()

print(f'Execution time: {time() - t0}')

Execution time: 0.09760713577270508


In [12]:
# check that the matrices are the same
A_wd - A_wd_

<Compressed Sparse Row sparse array of dtype 'int64'
	with 6189493 stored elements and shape (28093, 28093)>

In [13]:
A_d - A_d_

<Compressed Sparse Row sparse array of dtype 'int64'
	with 6167264 stored elements and shape (28093, 28093)>

In [14]:
A - A_

<Compressed Sparse Row sparse array of dtype 'int64'
	with 12282220 stored elements and shape (28093, 28093)>